In [1]:
import random
from collections import deque as dq

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
# Modules needed to transform Mario_env to fit ML Model
from env_transform import *
# Gym is an OpenAI toolkit for RL
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace
# Super Mario environment for OpenAI Gym
import gym_super_mario_bros

In [4]:
# Initialize Super Mario environment
env = gym_super_mario_bros.make("SuperMarioBros-1-1-v0")
# Limit the action-space to
#   0. walk right
#   1. jump right
env = JoypadSpace(env, [["right"], ["right", "A"]])


# Apply Wrappers to environment
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env)
env = ResizeObservation(env, shape=84)
# env = FrameStack(env, num_stack=4)

# env.reset()
# next_state, reward, done, info = env.step(action=1)
# print(f"{next_state.shape},\n {reward},\n {done},\n {info}")

In [5]:
# NN를 학습시키기 위한 hyperparameter
learning_rate = 0.0005
batch_size = 32
gamma = 0.98
buffer_limit = 100000

gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
# 강화학습은 Training data set이라는게 따로 없다. Agent가 행동을 취하고 데이터셋을 쌓아나가야합니다.
# 그 데이터셋을 쌓기 위한 버퍼
class ReplayBuffer():
    def __init__(self):
        self.buffer = dq(maxlen=buffer_limit)
    
    # 버퍼에는 (state, action ,reward, nstate, done) 값이 들어갑니다.
    def put(self, transition):
        self.buffer.append(transition)
    
    # 샘플 함수를 만드는 이유는 버퍼에 쌓인 데이터셋에서 랜덤으로 학습을 시키기 위함입니다.
    # 그냥 연속해서 쌓인 n개의 데이터셋을 그대로 사용하면 데이터간의 상관관계가 너무 크기 때문에 학슴이 잘 안됩니다.
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
           
        return torch.tensor(s_lst, dtype=torch.float, device=gpu), torch.tensor(a_lst, device=gpu), \
               torch.tensor(r_lst, dtype=torch.float, device=gpu), torch.tensor(s_prime_lst, dtype=torch.float, device=gpu), \
               torch.tensor(done_mask_lst, dtype=torch.float, device=gpu)
    
    def size(self):
        return len(self.buffer)

In [7]:
# Mario의 state가 (4, 84, 84)sahpe, action은 2개이기 때문에 input 3, output 2인 CNN생성
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1,out_channels = 32,kernel_size = 8,stride = 4)
        self.conv2 = nn.Conv2d(in_channels = 32,out_channels = 64,kernel_size = 4,stride = 2)
        self.conv3 = nn.Conv2d(in_channels = 64,out_channels = 64,kernel_size = 3,stride = 1)
        self.fc1 = nn.Linear(5184, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        x = x.to(gpu)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(-1,5184)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x.float()
    
    # epsilon greedy 전략을 사용합니다.
    # 간단하게 설명하면 탐험이라는 개념을 통해서 가보지 않은 경로를 가볼 수 있게 해줍니다.
    def sample_action(self, observation, epsilon):
        out = self.forward(observation)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()

In [8]:
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        
        # 벨만함수로부터 유도된 DQN 비용함수를 구현 학습시킵니다.
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [9]:
# Double Deep Q Learning 개념
# target_net을 semi constant로 사용
q = Qnet().to(gpu)
q_target = Qnet().to(gpu)
q_target.load_state_dict(q.state_dict())
memory = ReplayBuffer()

print_interval = 20
score = 0.0  
optimizer = optim.Adam(q.parameters(), lr=learning_rate)

for n_epi in range(40+1):
    print(f'epi : {n_epi}')
    epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
    s = env.reset()
    done = False

    while not done:
        a = q.sample_action((s.unsqueeze(0).unsqueeze(0)).float(), epsilon)
        s_prime, r, done, info = env.step(a)
        done_mask = 0.0 if done else 1.0
#         print(s.unsqueeze(0).shape, type(s.unsqueeze(0)))
#         print(a, type(a))
#         print(r, type(r))
#         print(s_prime.unsqueeze(0).shape, type(s_prime.unsqueeze(0)))
#         print(done_mask, type(done_mask))
        memory.put((s.unsqueeze(0).numpy(),a,r/100.0,s_prime.unsqueeze(0).numpy(), done_mask))
        s = s_prime

        score += r
        if done:
            break
    
    # 메모리가 어느정도 차야 random sample이 가능하기 때문에 일정 이상 차면 학습을 진행
    if memory.size()>2000:
        train(q, q_target, memory, optimizer)

    if n_epi%print_interval==0 and n_epi!=0:
        torch.save(q.state_dict(), 'Mario_weight.pt')
        f = open('./reward.txt', 'a')
        f.write("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%\n".format(
                                                        n_epi, score/print_interval, memory.size(), epsilon*100))
        f.close()
        # 일정 주기마다 semi constant인 target-net도 업데이트.
        q_target.load_state_dict(q.state_dict())
        print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                        n_epi, score/print_interval, memory.size(), epsilon*100))
        score = 0.0
env.close()

epi : 0
epi : 1
epi : 2
epi : 3
epi : 4
epi : 5
epi : 6
epi : 7
epi : 8
epi : 9
epi : 10
epi : 11
epi : 12
epi : 13
epi : 14
epi : 15
epi : 16
epi : 17
epi : 18
epi : 19
epi : 20
n_episode :20, score : 396.2, n_buffer : 9178, eps : 7.9%
epi : 21
epi : 22
epi : 23
epi : 24
epi : 25
epi : 26
epi : 27
epi : 28
epi : 29
epi : 30
epi : 31
epi : 32
epi : 33
epi : 34
epi : 35
epi : 36
epi : 37
epi : 38
epi : 39
epi : 40
n_episode :40, score : 428.6, n_buffer : 24413, eps : 7.8%
